# **COVID Dashboard**

# 1\. Contexto



*   **Dashboard;**
*   Google Data Studio: [GoogleDataStudio](https://lookerstudio.google.com/u/0/reporting/ff37666f-dc7e-4e82-abcd-5a3795cb7d3e/page/IjMfD)
*   **Fontes**;
*   Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
*   Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv))






#1.2. Dados


Os dados sobre **casos da COVID-19** são compilados pelo centro de ciência de sistemas e engenharia da universidade americana **John Hopkins** ([link](https://www.jhu.edu)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Os dados sobre **vacinação da COVID-19** são compilados pelo projeto Nosso Mundo em Dados (*Our World in Data* ou OWID) da universidade britânica de **Oxford** ([link](https://www.ox.ac.uk)). Os dados são **atualizados diariamente** deste janeiro de 2020 com uma **granularidade temporal de dias e geográfica de países**. O website do projeto pode ser acessado neste [link](https://ourworldindata.org) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## 2\. Pacotes e bibliotecas

In [ ]:
# Pacotes utilizados para processar os dados brutos, prezando adequar os dados para um painel de exploração interativa de dados.
import requests
import csv
import math
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from typing import Iterator
from pandas._libs.tslibs import period


## 3\. Extração

Fazendo a extração dos dados de casos e de vacinação. Já estou formatando os dados de casos para as datas que irei utilizar.

In [ ]:
# código de extração dos dados de casos:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv'
casos_1 = 'casos_1.csv'
response = requests.get(url)
with open(casos_1, 'w') as file:
    file.write(response.text)

casos = pd.read_csv(casos_1, sep=',')

# Função para criar um range de datas:
def data_range(data_inicial: datetime, data_final: datetime) -> Iterator[datetime]:
  data_range_dias: int = (data_final - data_inicial).days
  for lag in range(data_range_dias):
    yield data_inicial + timedelta(lag)

data_inicial = datetime(2021, 1, 1)
data_final = datetime(2021, 12, 31)

casos = None
casos_vazio = True

# Loop para coletar dados de diferentes datas:
for data in data_range(data_inicial=data_inicial, data_final=data_final):
  data_str = data.strftime('%m-%d-%Y')
  data_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{data_str}.csv'

  case = pd.read_csv(data_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(data.strftime('%Y-%m-%d'))

  if casos_vazio:
    casos = case
    casos_vazio = False
  else:
    casos = casos.append(case, ignore_index=True)


In [ ]:
# Observando o estado do Rio Grande do Sul, a fim de saber como as colunas se comportam:
casos.query('Province_State == "Rio Grande do Sul"').head()

In [ ]:
# - Wrangling da estrutura casos:
casos.head()

In [ ]:
casos.shape

In [ ]:
casos.info()

In [ ]:
# código de extração dos dados de vacinação;
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
vacinas_1 = 'vacinas.csv'
response = requests.get(url)

with open(vacinas_1, 'w') as file:
    file.write(response.text)


vacinas = pd.read_csv(vacinas_1, sep=',', parse_dates=[3], infer_datetime_format=True)

In [ ]:
# - Wrangling da estrutura vacinas:
vacinas.head()

In [ ]:
vacinas.shape

In [ ]:
vacinas.info()

## 4\. Transformação

Transformando os dados, passando-os para português, corrigindo a acentuação dos estados brasileiros, formatando as datas para uso no dashboard. Além disso, farei um cálculo da população total e analisarei a tendência de casos e mortes. Confirmarei a precisão dos dados para uso no dashboard e seguirei os mesmos passos para os dados de vacinação, organizando-os para uma melhor compreensão e verificando se estão conforme o esperado.

In [ ]:
# Renomeando os dados para português:
casos = casos.rename(
    columns={
        'Province_State': 'estado',
        'Country_Region': 'país',
        'Confirmed': 'confirmação',
        'Incident_Rate': 'taxa_de_incidentes',
        'Deaths': 'mortes',
        'Date': 'data'
    }
)
for col in casos.columns:
  casos = casos.rename(columns={col: col.lower()})


In [ ]:
estados_mapa = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Maranhao': 'Maranhão',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

casos['estado'] = casos['estado'].apply(lambda estado: estados_mapa.get(estado) if estado in estados_mapa.keys() else estado)


In [ ]:
# Tratando o dado para informações sobre mês e ano
casos['mês'] = casos['data'].apply(lambda data: data.strftime('%Y-%m'))
casos['ano'] = casos['data'].apply(lambda data: data.strftime('%Y'))


In [ ]:
# Adicionando coluna de população e removendo coluna desnecessárias:
casos['população'] = round(100000 * (casos['confirmação'] / casos['taxa_de_incidentes']))
casos = casos.drop('taxa_de_incidentes', axis=1)


In [ ]:
casos_ = None
casos_vazio = True

# Definindo a tendência para usar nas bases de casos e mortes:
def get_trend(rate: float) -> str:
  if np.isnan(rate):
    return np.NaN

  if rate < 0.85:
    status = 'descendente'
  elif rate > 1.15:
    status = 'ascendente'
  else:
    status = 'estável'

  return status

# Loop para análise por estado:
for estado in casos['estado'].drop_duplicates():
  casos_por_estado = casos.query(f'estado == "{estado}"').reset_index(drop=True)
  casos_por_estado = casos_por_estado.sort_values(by=['data'])

  casos_por_estado['confirmação_1d'] = casos_por_estado['confirmação'].diff(periods=1)
  casos_por_estado['confirmação_média_móvel_7d'] = np.ceil(casos_por_estado['confirmação_1d'].rolling(window=7).mean())
  casos_por_estado['confirmação_média_móvel_7d_relação_14d'] = casos_por_estado['confirmação_média_móvel_7d']/casos_por_estado['confirmação_média_móvel_7d'].shift(periods=14)
  casos_por_estado['tendência_confirmada'] = casos_por_estado['confirmação_média_móvel_7d_relação_14d'].apply(get_trend)

  casos_por_estado['mortes_1d'] = casos_por_estado['mortes'].diff(periods=1)
  casos_por_estado['mortes_média_móvel_7d'] = np.ceil(casos_por_estado['mortes_1d'].rolling(window=7).mean())
  casos_por_estado['mortes_média_móvel_7d_relação_14d'] = casos_por_estado['mortes_média_móvel_7d']/casos_por_estado['mortes_média_móvel_7d'].shift(periods=14)
  casos_por_estado['tendência_mortes'] = casos_por_estado['mortes_média_móvel_7d_relação_14d'].apply(get_trend)

  if casos_vazio:
    casos_ = casos_por_estado
    casos_vazio = False
  else:
    casos_ = casos_.append(casos_por_estado, ignore_index=True)

casos = casos_
casos_ = None


In [ ]:
# Transformando os dados para o tipo que desejo:
casos['população'] = casos['população'].astype('Int64')
casos['confirmação_1d'] = casos['confirmação_1d'].astype('Int64')
casos['confirmação_média_móvel_7d'] = casos['confirmação_média_móvel_7d'].astype('Int64')
casos['mortes_1d'] = casos['mortes_1d'].astype('Int64')
casos['mortes_média_móvel_7d'] = casos['mortes_média_móvel_7d'].astype('Int64')


In [ ]:
# Organizando os dados para melhor visualização:
casos = casos[['data', 'país', 'estado', 'população', 'confirmação', 'confirmação_1d', 'confirmação_média_móvel_7d', 'confirmação_média_móvel_7d_relação_14d', 'tendência_confirmada', 'mortes', 'mortes_1d', 'mortes_média_móvel_7d', 'mortes_média_móvel_7d_relação_14d', 'tendência_mortes', 'mês', 'ano']]


In [ ]:
# Observando o dado para ter melhor entendimento de como ficaram:
casos.head(25)

In [ ]:
casos.shape

In [ ]:
casos.info()

In [ ]:
# Filtrando o dado pelo País e selecionando colunas:

vacinas = vacinas.query('location == "Brazil"').reset_index(drop=True)
vacinas = vacinas[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]


In [ ]:
# Preenchendo valores nulos e filtrando o dado pelo ano proposto:
vacinas = vacinas.fillna(method='ffill')

In [ ]:
vacinas = vacinas[(vacinas['date'] >= '2021-01-01') & (vacinas['date'] <= '2021-12-31')].reset_index(drop=True)


In [ ]:
# Renomeando os dados para português:
vacinas = vacinas.rename(
    columns={
        'location' : 'país',
        'total_vaccinations': 'total',
        'people_vaccinated': 'primeira_dose',
        'people_fully_vaccinated': 'segunda_dose',
        'total_boosters': 'terceira_dose',
        'date': 'data',
        'population': 'população'
    }
)


In [ ]:
# Tratando o dado para informações sobre mês e ano, adicionando um cálculo sobre porcentagens:
vacinas['mês'] = vacinas['data'].apply(lambda data: data.strftime('%Y-%m'))
vacinas['ano'] = vacinas['data'].apply(lambda data: data.strftime('%Y'))


In [ ]:
vacinas['porcentagem_primeira_dose'] = round(vacinas['primeira_dose'] / vacinas['população'], 4)
vacinas['porcentagem_segunda_dose'] = round(vacinas['segunda_dose'] / vacinas['população'], 4)
vacinas['porcentagem_terceira_dose'] = round(vacinas['terceira_dose'] / vacinas['população'], 4)


In [ ]:
# Transformando os dados para o tipo que desejo:
vacinas['população'] = vacinas['população'].astype('Int64')
vacinas['total'] = vacinas['total'].astype('Int64')
vacinas['primeira_dose'] = vacinas['primeira_dose'].astype('Int64')
vacinas['segunda_dose'] = vacinas['segunda_dose'].astype('Int64')
vacinas['terceira_dose'] = vacinas['terceira_dose'].astype('Int64')


In [ ]:
# Organizando os dados para melhor visualização:
vacinas = vacinas[['data', 'país', 'população', 'total', 'primeira_dose', 'porcentagem_primeira_dose', 'segunda_dose', 'porcentagem_segunda_dose', 'terceira_dose', 'porcentagem_terceira_dose', 'mês', 'ano']]


In [ ]:
# Observando o dado para ter melhor entendimento de como ficaram:
vacinas.head()

In [ ]:
vacinas.tail()

In [ ]:
vacinas.shape

In [ ]:
vacinas.info()

## 5\. Carregamento

Criando os arquivos "casos-covid.csv" e "vacinas-covid.csv" para serem usados como dados do dashboard.

In [ ]:
# Salvando os dados processados de casos em formato CSV:
casos.to_csv('./casos-covid.csv', sep=',', index=False)

In [ ]:
# Salvando os dados processados de vacinação em formato CSV:
vacinas.to_csv('./vacinas-covid.csv', sep=',', index=False)

## 6\. Storytelling

Em 2021, o Brasil enfrentou uma crise devido à pandemia de COVID-19. A vacinação começou lentamente no final de janeiro, mas o ritmo era insuficiente para conter a propagação do vírus. A situação continuou crítica ao longo dos meses seguintes, com um aumento constante no número de casos e mortes, apesar do início da administração da segunda dose da vacina em fevereiro. A propagação do vírus se acelerou em março, evidenciando a urgência de acelerar o programa de vacinação. Foi somente em junho que o ritmo da vacinação começou a se intensificar, e a partir de julho, finalmente começamos a observar uma redução nos casos e mortes, trazendo um sinal de alívio e esperança para o país.